In [1]:
import cv2
import numpy as np

In [34]:
img = cv2.imread("images/man.jpg")
# cap = cv2.VideoCapture(0)

# while cap.isOpened():
#     # 이미지 읽어오기 : (ret : 성공 여부, frame : 이미지)
#     ret, frame = cap.read()
#     i += 1
#     print(ret, i)
#     # 이미지 절반으로 줄이기
#     frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_AREA)
#     # 캠화면을 끄기 위한 방법
#     if cv2.waitKey(1) == 27: # esc키로 종료
#         break
#     cv2.imshow("WEB CAM", frame)

# cap.release() # 외부장치 접근 해제
# cv2.destroyAllWindows()

# 그레이 스케일로 변환 : 단순하게 만들기 위함
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 노이즈 제거 작업(가우시안 필터 : 라플라시안 필터를 사용하려면 블러가 되야하기 때문에 쓴다.)
# 라플라시안 필터 사용하기 전에 항상 필수적으로 사용
img_gray = cv2.GaussianBlur(img_gray, (9,9), 0)

#라플라시안 필터 적용
edge = cv2.Laplacian(img_gray, -1, None, 5)

# 전역 스레시홀딩을 사용해서 경계값을 뚜렷하게 해줌
ret, sketch = cv2.threshold(edge, 30, 255, cv2.THRESH_BINARY_INV)

# 경계값을 강조하는 팽창 연산
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
sketch_2 = cv2.erode(sketch, kernel)

# 경계값을 자연스럽게하는 미디언 블러 사용
sketch = cv2.medianBlur(sketch_2, 5)

# 이미지의 사이즈를 3차원으로 변경
img_sketch = cv2.cvtColor(sketch, cv2.COLOR_GRAY2BGR)

# 이미지에 컬러 적용
img_paint = cv2.blur(img, (14, 14))
img_paint = cv2.bitwise_and(img_paint, img_paint, mask=sketch)


merged = np.hstack((img_sketch, img_paint))
cv2.imshow("IMAGE", merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

틀린 그림 찾기

In [2]:
import cv2
import numpy as np

img = cv2.imread('diff/diff_3.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
re_img = cv2.resize(img, None, fx=0.7, fy=0.7, interpolation=cv2.INTER_AREA)

cv2.imshow("diff", re_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 이미지 나눠주기
# 이미지 합치기(경계값이나 바이너리 이미지 찾기)
# 다른 부분의 위치값을 표시하기

In [68]:
h = int(re_img.shape[0]/2)

img_1 = re_img[:h, :]
img_2 = re_img[h:, :]
cv2.imshow("diff", img_1)
cv2.imshow("diff2", img_2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [61]:
# 123번부터 497번 인덱스까지가 이미지
np.where((img_1 == 0).sum(axis=1) <= 10)

(array([123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
        136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
        149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
        162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174,
        175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187,
        188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200,
        201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213,
        214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226,
        227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
        240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252,
        253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265,
        266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278,
        279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291,
        292, 293, 294, 295, 296, 297, 298, 299, 300

In [65]:
# 이미지에서 검정색 배경을 제거하는 작업
ht = np.where((img_1 == 0).sum(axis=1) <= 50)[0][0]
hb = np.where((img_1 == 0).sum(axis=1) <= 50)[0][-1]

In [67]:
new_img = re_img[ht:hb]
cv2.imshow("diff", new_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [40]:
# 재윤씨 아이디어
def img_rec(path):
    
    # 이미지 불러오기 & 이미지 크기 조정
    img = cv2.imread(path)
    re_img = cv2.resize(img, None, fx=0.7, fy=0.7, interpolation=cv2.INTER_AREA)
    
    # 이미지 처리를 쉽게 하기 위해서 gray로 변환

    
    # 이미지 두개로 분할
    h = int(re_img.shape[0]/2)
    img_1 = re_img[:h, :]
    img_2 = re_img[h:, :]

    # 이미지에서 검정색 배경을 제거하는 작업
    imgs = [img_1, img_2]
    crop_imgs = []
    for i in range(2):
        img_gray = cv2.cvtColor(imgs[i], cv2.COLOR_BGR2GRAY)
        ht = np.where((img_gray == 0).sum(axis=1) <= 50)[0][0]
        hb = np.where((img_gray == 0).sum(axis=1) <= 50)[0][-1]
        crop_imgs.append(imgs[i][ht:hb])
    return tuple(crop_imgs)
# 보관된 이미지들
img_source = ['diff_1.jpg', 'diff_2.jpg','diff_3.jpg', 'diff_4.jpg','diff_5.jpg']
# 보관된 이미지를 반복해서 수정
for i in range(len(img_source)):
    # 이미지를 분할하는 img_rec 사용자 정의 함수를 이용
    img1, img2 = img_rec('diff/' + img_source[i])
    # 이미지 크기(height)가 다르면 마지막 한줄을 삭제
    if img1.shape[0] != img2.shape[0]:
        img2 = img2[:-1,:,:]
    # 분할된 이미지를 저장하는 코드
    cv2.imwrite('diff/' + f'upper_{i}.jpg', img1)
    cv2.imwrite('diff/' + f'bottom_{i}.jpg', img2)

img1, img2 = img_rec('diff/diff_0.jpg')
cv2.imshow("diff1", img1)
cv2.imshow("diff2", img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [67]:
# 이미지 차이 계산식
img1 = cv2.imread('diff/upper_4.jpg')
img2 = cv2.imread('diff/bottom_4.jpg')
print(img1.shape, img2.shape)

(375, 466, 3) (375, 466, 3)


In [68]:
cv2.imshow("diff1", img1)
cv2.imshow("diff2", img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [123]:
# 이미지 차이 계산 식
aaa = 1

img1 = cv2.imread('diff/upper_'+ str(aaa) +'.jpg')
img2 = cv2.imread('diff/bottom_'+ str(aaa) +'.jpg')

diff = cv2.absdiff(img1, img2)
cv2.imshow("diff_b", diff)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [127]:
gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

# 차이강조를 위해서 스레시홀드를 진행
# 변수 두개 설정해 줘야함(thresh만 변수선언하면 에러남)
_, thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)

## 컨투어 찾기 (틀린 그림 좌표 찾기)
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# print(contours)
print(len(contours))

# 이미지 차이 계산식
aaa = 1

img1 = cv2.imread('diff/upper_'+ str(aaa) +'.jpg')
img2 = cv2.imread('diff/bottom_'+ str(aaa) +'.jpg')

# 컨투어 찾기
for cnt in contours:
    # 컨투어를 감싸는 최소 사각형 계산
    x, y, w, h = cv2.boundingRect(cnt)
    if w>2 and h>2:
        cv2.rectangle(img1, (x,y), (x+w, y+h), (255,255,255), 3)

diff = cv2.absdiff(img1, img2)
gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours:
    # 컨투어를 감싸는 최소 사각형 계산
    x, y, w, h = cv2.boundingRect(cnt)
    if w>2 and h>2:
        cv2.rectangle(img1, (x,y), (x+w, y+h), (255,255,255), 3)


cv2.imshow("diff_b", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

6


In [196]:
def diff_img(k):
    # 번호에 맞는 이미지 가져오기
    img1 = cv2.imread(f'diff/upper_{int(k)}.jpg')
    img2 = cv2.imread(f'diff/bottom_{int(k)}.jpg')

    # 차이값 계산
    diff = cv2.absdiff(img1, img2)

    # 스레시홀딩
    # 2차원으로 변환
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

    # 차이강조를 위해서 스레시홀드를 진행
    # 변수 두개 설정해 줘야함(thresh만 변수선언하면 에러남)
    _, thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)
    
    # 컨투어 찾기 (틀린 그림 좌표 찾기)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 컨투어 찾기
    for cnt in contours:
        # 컨투어를 감싸는 최소 사각형 계산
        x, y, w, h = cv2.boundingRect(cnt)
        if w>3 and h>3:
            cv2.rectangle(img1, (x,y), (x+w, y+h), (255,255,255), 10)

    # 두번째 diff 작업
    # 번호에 맞는 이미지 가져오기
    img_after = img1.copy()
    img2 = cv2.imread(f'diff/bottom_{int(k)}.jpg')

    # 차이값 계산
    diff = cv2.absdiff(img_after, img2)

    # 스레시홀딩
    # 2차원으로 변환
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

    # 차이강조를 위해서 스레시홀드를 진행
    # 변수 두개 설정해 줘야함(thresh만 변수선언하면 에러남)
    _, thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)
    
    # 컨투어 찾기 (틀린 그림 좌표 찾기)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    real_img = cv2.imread(f'diff/upper_{int(k)}.jpg')
    # 컨투어 찾기
    for cnt in contours:
        # 컨투어를 감싸는 최소 사각형 계산
        x, y, w, h = cv2.boundingRect(cnt)
        if w>5 and h>5:
            cv2.rectangle(real_img, (x,y), (x+w, y+h), (255,0,0), 2)

    return real_img

img = diff_img(4)
cv2.imshow("diff_b", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [203]:
## 1. 이미지 분할 코드

# 보관된 이미지들
img_source = ['diff_1.jpg', 'diff_2.jpg','diff_3.jpg', 'diff_4.jpg','diff_5.jpg']
# 보관된 이미지를 반복해서 수정
for i in range(len(img_source)):
    # 이미지를 분할하는 img_rec 사용자 정의 함수를 이용
    img1, img2 = img_rec('diff/' + img_source[i])
    # 이미지 크기(height)가 다르면 마지막 한줄을 삭제
    if img1.shape[0] != img2.shape[0]:
        img2 = img2[:-1,:,:]
    # 분할된 이미지를 저장하는 코드
    cv2.imwrite('diff/' + f'upper_{i}.jpg', img1)
    cv2.imwrite('diff/' + f'bottom_{i}.jpg', img2)

# 2. 틀린 부분 찾고 그림 그리기
# 2-1. 처음에는 img1 과 img2의 diff를 만들고
# 2-2. 차이값에 대한 img1의 사각형을 그림
# 2-3. 사각형의 두께가 크면 클수록 사각형 겹침이 생김
# 2-4. 2-3에서 만든 img1에 다시 한번 diff작업을 진행
# 2-5. 더 큰 사각형을 만들어줌
def diff_img(k):
    # 번호에 맞는 이미지 가져오기
    img1 = cv2.imread(f'diff/upper_{int(k)}.jpg')
    img2 = cv2.imread(f'diff/bottom_{int(k)}.jpg')

    # 차이값 계산
    diff = cv2.absdiff(img1, img2)

    # 스레시홀딩
    # 2차원으로 변환
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

    # 차이강조를 위해서 스레시홀드를 진행
    # 변수 두개 설정해 줘야함(thresh만 변수선언하면 에러남)
    _, thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)
    
    # 컨투어 찾기 (틀린 그림 좌표 찾기)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 컨투어 찾기
    for cnt in contours:
        # 컨투어를 감싸는 최소 사각형 계산
        x, y, w, h = cv2.boundingRect(cnt)
        if w>3 and h>3:
            cv2.rectangle(img1, (x,y), (x+w, y+h), (255,255,255), 10)
    # 사각형이 그려진 이미지를 가지고 더 큰 사각형을 그리는 작업
    # 두번째 diff 작업
    # 번호에 맞는 이미지 가져오기
    img_after = img1.copy()
    img2 = cv2.imread(f'diff/bottom_{int(k)}.jpg')

    # 차이값 계산
    diff = cv2.absdiff(img_after, img2)

    # 스레시홀딩
    # 2차원으로 변환
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

    # 차이강조를 위해서 스레시홀드를 진행
    # 변수 두개 설정해 줘야함(thresh만 변수선언하면 에러남)
    _, thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)
    
    # 컨투어 찾기 (틀린 그림 좌표 찾기)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    real_img = cv2.imread(f'diff/upper_{int(k)}.jpg')
    # 컨투어 찾기
    for cnt in contours:
        # 컨투어를 감싸는 최소 사각형 계산
        x, y, w, h = cv2.boundingRect(cnt)
        if w>5 and h>5:
            cv2.rectangle(real_img, (x,y), (x+w, y+h), (255,0,0), 2)

    return real_img, img2

input_num = input("0~4까지의 숫자를 입력하시오.")
img_1, img_2 = diff_img(input_num)
original_u = cv2.imread(f'diff/upper_{int(input_num)}.jpg')
original_b = cv2.imread(f'diff/bottom_{int(input_num)}.jpg')
merged = np.hstack((original_u, original_b))
cv2.imshow("틀린 그림 찾기 원본 이미지", merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

merged = np.hstack((img_1, img_2))
cv2.imshow("틀린 그림 찾기 d", merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

0~4까지의 숫자를 입력하시오. 0
